In [1]:
from clickhouse_driver import Client
client = Client(host='localhost')

In [2]:
# вывод списка баз данных
client.execute('SHOW DATABASES')

[('_temporary_and_external_tables',), ('default',), ('system',)]

In [3]:
# создание базы данных example в кластере company_cluster
client.execute('CREATE DATABASE IF NOT EXISTS example ON CLUSTER company_cluster')

[('clickhouse-node1', 9000, 0, '', 3, 0),
 ('clickhouse-node3', 9000, 0, '', 2, 0),
 ('clickhouse-node2', 9000, 0, '', 1, 0),
 ('clickhouse-node4', 9000, 0, '', 0, 0)]

In [4]:
client.execute('SHOW DATABASES')

    

[('_temporary_and_external_tables',), ('default',), ('example',), ('system',)]

In [5]:
# создание таблицы regular_table
client.execute('CREATE TABLE example.regular_table ON CLUSTER company_cluster (id Int64, x Int32) Engine=MergeTree() ORDER BY id')

[('clickhouse-node1', 9000, 0, '', 3, 0),
 ('clickhouse-node3', 9000, 0, '', 2, 0),
 ('clickhouse-node4', 9000, 0, '', 1, 0),
 ('clickhouse-node2', 9000, 0, '', 0, 0)]

In [6]:
# запись данных в таблицу
client.execute('INSERT INTO example.regular_table (id, x) VALUES (1, 10), (2, 20)')

[]

In [7]:
# вывод данных
client.execute('SELECT * FROM example.regular_table')

[(1, 10), (2, 20)]

создание дистрибутивной реплицированной таблицы:

- зайти в контейнер первого шарда '''$ docker exec -it clickhouse-node1 bash'''
- запустить Клиент командной строки '''$ clickhouse-client'''
- выполнить команды:
    - CREATE DATABASE shard;

    - CREATE DATABASE replica;

    - CREATE TABLE shard.test (id Int64, event_time DateTime) Engine=ReplicatedMergeTree('/clickhouse/tables/shard1/test', 'replica_1') PARTITION BY toYYYYMMDD(event_time) ORDER BY id;

    - CREATE TABLE replica.test (id Int64, event_time DateTime) Engine=ReplicatedMergeTree('/clickhouse/tables/shard2/test', 'replica_2') PARTITION BY toYYYYMMDD(event_time) ORDER BY id;

    - CREATE TABLE default.test (id Int64, event_time DateTime) ENGINE = Distributed('company_cluster', '', test, rand());

- зайти в контейнер первого шарда '''$ docker exec -it clickhouse-node3 bash'''
- запустить Клиент командной строки '''$ clickhouse-client'''
- выполнить команды:
    - CREATE DATABASE shard;

    - CREATE DATABASE replica;

    - CREATE TABLE shard.test (id Int64, event_time DateTime) Engine=ReplicatedMergeTree('/clickhouse/tables/shard2/test', 'replica_1') PARTITION BY toYYYYMMDD(event_time) ORDER BY id;

    - CREATE TABLE replica.test (id Int64, event_time DateTime) Engine=ReplicatedMergeTree('/clickhouse/tables/shard1/test', 'replica_2') PARTITION BY toYYYYMMDD(event_time) ORDER BY id;

    - CREATE TABLE default.test (id Int64, event_time DateTime) ENGINE = Distributed('company_cluster', '', test, rand());


In [8]:
# Запись данных в дистрибутивную таблицу
client.execute('INSERT INTO default.test (id, event_time) VALUES (1, today()), (2, today()), (3, now())')

[]

In [9]:
# вывод данных из таблицы реплики
client.execute('SELECT * FROM replica.test') 

[(1, datetime.datetime(2022, 11, 26, 0, 0)),
 (3, datetime.datetime(2022, 11, 26, 4, 20, 8))]

In [10]:
# вывод данных из таблицы реплики
client.execute('SELECT * FROM shard.test') 

[(2, datetime.datetime(2022, 11, 26, 0, 0))]

In [11]:
# вывод всех данных из дистрибутивной таблицы
print(client.execute('SELECT * FROM default.test') )

[(1, datetime.datetime(2022, 11, 26, 0, 0)), (3, datetime.datetime(2022, 11, 26, 4, 20, 8)), (2, datetime.datetime(2022, 11, 26, 0, 0))]
